In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         os.path.join(dirname, filename)

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Modules

In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf
import os
import cv2
from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt
import random

from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop,Adam
from sklearn.model_selection import train_test_split
import tensorflow as tf
print(tf.__version__)

# Loading Data

In [ ]:
BASE_DIR=('../input/state-farm-distracted-driver-detection/imgs/')
train_dir=os.path.join(BASE_DIR,'train/')
test_dir=os.path.join(BASE_DIR,'test/')

print('Number of images in training set = ',str(len(glob(train_dir+'*/*'))))
print('Number of images in testing set = ',str(len(glob(test_dir+'*'))))

## Making Train, validation and test directories

In [ ]:
class_labels=['safe driving','texting - right','talking on the phone - right','texting - left','talking on the phone - left','operating the radio','drinking','reaching behind','hair and makeup','talking to passenger']

#training directories
for label in class_labels:
    tf.io.gfile.makedirs('/kaggle/working/train_dataset/'+label+'/')
    
#validation directories
for label in class_labels:
    tf.io.gfile.makedirs('/kaggle/working/val_dataset/'+label+'/')
    
#test directories
for label in class_labels:
    tf.io.gfile.makedirs('/kaggle/working/test_dataset/'+label+'/')

    


In [ ]:
os.listdir("/kaggle/working/train_dataset/")

### The predicted 10 classes
* c0: safe driving
* c1: texting - right
* c2: talking on the phone - right
* c3: texting - left
* c4: talking on the phone - left
* c5: operating the radio
* c6: drinking
* c7: reaching behind
* c8: hair and makeup
* c9: talking to passenger

In [ ]:
SAFE_DRIVING=os.path.join(train_dir,'c0/')
TEXTING_RIGHT=os.path.join(train_dir,'c1/')
TALKING_ON_PHONE_RIGHT=os.path.join(train_dir,'c2/')
TEXTING_LEFT=os.path.join(train_dir,'c3/')
TALKING_ON_PHONE_LEFT=os.path.join(train_dir,'c4/')
OPERATING_THE_RADIO=os.path.join(train_dir,'c5/')
DRINKING=os.path.join(train_dir,'c6/')
REACHING_BEHIND=os.path.join(train_dir,'c7/')
HAIR_MAKEUP=os.path.join(train_dir,'c8/')
TALKING_TO_PASSENGER=os.path.join(train_dir,'c9/')

print("Safe driving = ",len(os.listdir(SAFE_DRIVING)))
print("Texting right = ",len(os.listdir(TEXTING_RIGHT)))
print("Talking on phone right = ",len(os.listdir(TALKING_ON_PHONE_RIGHT)))
print("Texting left = ",len(os.listdir(TEXTING_LEFT)))
print("Talking on phone left = ",len(os.listdir(TALKING_ON_PHONE_LEFT)))
print("Operating the radio = ",len(os.listdir(OPERATING_THE_RADIO)))
print("Drinking = ",len(os.listdir(DRINKING)))
print("Reaching behind = ",len(os.listdir(REACHING_BEHIND)))
print("Hair makeup = ",len(os.listdir(HAIR_MAKEUP)))
print("Talking to passenger = ",len(os.listdir(TALKING_TO_PASSENGER)))

In [ ]:
# Distribution of images in respective directories

import pathlib 
import cv2
import shutil

def distribution(param1, param2, param3):

#     data_dir = pathlib.Path(param2)
#     data_dir

    a=os.listdir(param3)

    test = a[:200]
    val = a[201:401]
    train = a[402:]


    for images in test:
#         print(f"../input/state-farm-distracted-driver-detection/imgs/train/{param1}/"+images, f"/kaggle/working/test_dataset/{param2}/")
        shutil.copy(f"../input/state-farm-distracted-driver-detection/imgs/train/{param1}/"+images, f"/kaggle/working/test_dataset/{param2}/")
    for images in val:
        shutil.copy(f"../input/state-farm-distracted-driver-detection/imgs/train/{param1}/"+images, f"/kaggle/working/val_dataset/{param2}/")

    for images in train:
        shutil.copy(f"../input/state-farm-distracted-driver-detection/imgs/train/{param1}/"+images, f"/kaggle/working/train_dataset/{param2}/")


    print(f"The count of images for test_dataset > {param2} ",len(os.listdir(f"/kaggle/working/test_dataset/{param2}")))
    print(f"The count of images for val_dataset > {param2} ",len(os.listdir(f"/kaggle/working/val_dataset/{param2}")))
    print(f"The count of images for train_dataset > {param2} ",len(os.listdir(f"/kaggle/working/train_dataset/{param2}")))
    
    


In [ ]:
dir_list = [SAFE_DRIVING, TEXTING_RIGHT, TALKING_ON_PHONE_RIGHT, TEXTING_LEFT, TALKING_ON_PHONE_LEFT, OPERATING_THE_RADIO, DRINKING, REACHING_BEHIND, HAIR_MAKEUP, TALKING_TO_PASSENGER]
i=0
for class_label in class_labels:
    print(f"c{i}")
    distribution(f"c{i}", class_label, dir_list[i]) 
    i+=1


**Starting with Image Preprocessing**

In [ ]:
import PIL
import pathlib

data_dir = "/kaggle/working/train_dataset/"

data_dir = pathlib.Path(data_dir)


print("The count of total images for training set ",len(list(data_dir.glob('*/*.jpg'))))


In [ ]:
driving = list(data_dir.glob('safe driving/*'))
driving[:5]

In [ ]:
import PIL

PIL.Image.open(str(driving[0]))

### Mapping

In [ ]:
all_training_images = {
    'safe driving' :  list(data_dir.glob('safe driving/*')),
    'texting - right' :  list(data_dir.glob('texting - right/*')),
    'talking on the phone - right' :  list(data_dir.glob('talking on the phone - right/*')),
    'texting - left' :  list(data_dir.glob('texting - left/*')),
    'talking on the phone - left' :  list(data_dir.glob('talking on the phone - left/*')),
    'operating the radio' :  list(data_dir.glob('operating the radio/*')),
    'drinking' :  list(data_dir.glob('drinking/*')),
    'reaching behind' :  list(data_dir.glob('reaching behind/*')),
    'hair and makeup' :  list(data_dir.glob('hair and makeup/*')),
    'talking to passenger' :  list(data_dir.glob('talking to passenger/*')),
    
}

### View safe driving image

In [ ]:
PIL.Image.open(all_training_images['safe driving'][1])

### Getting count of each directory

In [ ]:
# Gives all the count of images in all the directories of training dataset


for class_label, img_count in all_training_images.items():
    print(class_label)
    print(len(img_count))

### Setting variable of path of all three directories

In [ ]:
updated_train_dir="/kaggle/working/train_dataset/"
updated_val_dir="/kaggle/working/val_dataset/"
updated_test_dir="/kaggle/working/test_dataset/"
print(os.listdir(updated_train_dir))
print(os.listdir(updated_val_dir))
print(os.listdir(updated_test_dir))

### Creating Image data generator 

In [ ]:
# train_datagen=ImageDataGenerator(rescale=1.0/255,
#                                  rotation_range=30,
#                                  width_shift_range=0.2,
#                                  height_shift_range=0.2,
#                                  zoom_range=0.2,
#                                  )

# val_datagen=ImageDataGenerator(rescale=1.0/255)

# test_datagen=ImageDataGenerator(rescale=1.0/255)

# train_generator=train_datagen.flow_from_directory(updated_train_dir,color_mode="grayscale",target_size=(180,180),class_mode='categorical')

# val_generator=val_datagen.flow_from_directory(updated_val_dir,color_mode="grayscale",target_size=(180,180),class_mode='categorical')

# test_generator=test_datagen.flow_from_directory(updated_test_dir,color_mode="grayscale",target_size=(180,180),class_mode='categorical')

### Creating models


In [ ]:
train_datagen3=ImageDataGenerator(rescale=1.0/255,
                                 rotation_range=30,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range=0.2,
                                 )

val_datagen3=ImageDataGenerator(rescale=1.0/255)

test_datagen3=ImageDataGenerator(rescale=1.0/255)
height = 200
width = 280
train_generator3=train_datagen3.flow_from_directory(updated_train_dir,target_size=(width,height),batch_size=128,class_mode='categorical')

val_generator3=val_datagen3.flow_from_directory(updated_val_dir,target_size=(width,height),batch_size=128,class_mode='categorical')

test_generator3=test_datagen3.flow_from_directory(updated_test_dir,target_size=(width,height),batch_size=128,class_mode='categorical')

In [ ]:
# from tensorflow.keras.applications.resnet50 import ResNet50
# from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
# pretrained_model3 = ResNet50(weights= 'imagenet', include_top=False, input_shape= (180,180,3))
# pretrained_model3 = VGG16(weights= 'imagenet', include_top=False, input_shape= (180,180,3))
#freazing the trained layers
# for layers in pretrained_model3.layers:
#     layers.trainable = False
# pretrained_model3.summary()


In [ ]:
model=tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu',padding='same', input_shape=(width, height, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy',
               metrics=['accuracy','Precision','Recall'])

In [ ]:
# last_layer=pretrained_model3.get_layer('block5_pool')
# last_output = last_layer.output

# x=tf.keras.layers.Flatten()(last_output)
# x=tf.keras.layers.Dense(2048,activation='relu')(x)
# x=tf.keras.layers.Dense(1024,activation='relu')(x)
# x=tf.keras.layers.Dropout(0.2)(x)
# x=tf.keras.layers.Dense(256,activation='relu')(x)
# x=tf.keras.layers.Dropout(0.2)(x)
# x=tf.keras.layers.Dense(10,activation='sigmoid')(x)
# # detect and init the TPU
# # tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # # instantiate a distribution strategy
# # tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
# # with tpu_strategy.scope():
# model3=tf.keras.Model(pretrained_model3.input,x)

# model3.compile(optimizer=RMSprop(lr=0.0001),
#               loss='categorical_crossentropy',
#                metrics=['accuracy','Precision','Recall'])

In [ ]:
history3=model.fit(train_generator3,validation_data=val_generator3,epochs=30,verbose=2)

In [ ]:
acc3 = history3.history['accuracy']
val_acc3 = history3.history['val_accuracy']

train_precision3=history3.history['precision']
val_precision3=history3.history['val_precision']

train_recall3=history3.history['recall']
val_recall3=history3.history['val_recall']

loss3 = history3.history['loss']
val_loss3 = history3.history['val_loss']
epochs = range(len(acc3))

plt.plot(epochs, acc3, 'r', label='Training accuracy')
plt.plot(epochs, val_acc3, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, train_precision3, 'r', label='Training precision')
plt.plot(epochs, val_precision3, 'b', label='Validation precision')
plt.title('Training and validation precision')
plt.legend()
plt.figure()

plt.plot(epochs, train_recall3, 'r', label='Training recall')
plt.plot(epochs, val_recall3, 'b', label='Validation recall')
plt.title('Training and validation recall')
plt.legend()
plt.figure()

plt.plot(epochs, loss3, 'r', label='Training Loss')
plt.plot(epochs, val_loss3, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
eval_result3 = model.evaluate_generator(test_generator3)
# print(eval_result3)
print('loss rate at evaluation data :', eval_result3[0])
print('accuracy rate at evaluation data :', eval_result3[1])

model.save('PNP.h5')

In [ ]:
import os
path =os.listdir("../input/state-farm-distracted-driver-detection")
path

In [ ]:
# model=tf.keras.models.Sequential([
#     # This is the first convolution
#     tf.keras.layers.Conv2D(16, (3,3), activation='relu',padding='same', input_shape=(180, 180, 1)),
#     tf.keras.layers.Flatten(),
#     # 512 neuron hidden layer
#     tf.keras.layers.Dense(512, activation='relu'),
#     # Only 1 output neuron. It will contain a value from 0-1 
#     tf.keras.layers.Dense(10,activation='softmax')
# ])

In [ ]:
# model.compile(optimizer=RMSprop(lr=0.001),
#               metrics=['accuracy','Precision','Recall'])

In [ ]:
# model.summary()

### Training a model

In [ ]:
# Ylabels=['drinking', 'safe driving', 'talking to passenger', 'reaching behind', 'operating the radio', 'talking on the phone - right', 'texting - left', 'hair and makeup', 'talking on the phone - left', 'texting - right']

In [ ]:
# history=model.fit(train_generator,epochs=30,verbose=2)

### Plotting

In [ ]:
# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']

# train_precision=history.history['precision']
# val_precision=history.history['val_precision']

# train_recall=history.history['recall']
# val_recall=history.history['val_recall']

# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(len(acc))

# plt.plot(epochs, acc, 'r', label='Training accuracy')
# plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
# plt.title('Training and validation accuracy')
# plt.legend()
# plt.figure()

# plt.plot(epochs, train_precision, 'r', label='Training precision')
# plt.plot(epochs, val_precision, 'b', label='Validation precision')
# plt.title('Training and validation precision')
# plt.legend()
# plt.figure()

# plt.plot(epochs, train_recall, 'r', label='Training recall')
# plt.plot(epochs, val_recall, 'b', label='Validation recall')
# plt.title('Training and validation recall')
# plt.legend()
# plt.figure()

# plt.plot(epochs, loss, 'r', label='Training Loss')
# plt.plot(epochs, val_loss, 'b', label='Validation Loss')
# plt.title('Training and validation loss')
# plt.legend()

# plt.show()


In [ ]:
# import cv2

# # Resizing the images
# X, Y = [], []


# for class_label, img_count in all_training_images.items():
#     print(class_label)
#     for image in img_count:
#         img = cv2.imread(str(image))
#         resized_img = cv2.resize(img,(180,180))
#         X.append(resized_img)
#         Y.append(class_label)

In [ ]:
# X = np.array(X)
# Y = np.array(Y)

### Creating model
